# Working with the Apostrophe VectorBD
The vector DB is currently hosted at Deep Lake - this is subject to change

Modify dependencies for your system needs - only needed once

In [ ]:
!python3 -m pip install --upgrade langchain 'deeplake[enterprise]' openai tiktoken

Enter the API key for OpenAI and define special embeddings for OpenAI

In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(disallowed_special=())


Point the `db` variable at the right database (subject to change)

In [7]:

from langchain.vectorstores import DeepLake

db = DeepLake(
    dataset_path=f"hub://bodonkey/apostrophe-docs-db-v2",
    read_only=True,
    embedding=embeddings,
)

Deep Lake Dataset in hub://bodonkey/apostrophe-docs-db-v2 already exists, loading from the storage


Modify retrieval patterns

In [8]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 15

Specify the langchain model interactions including additional prompt engineering

In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Define the custom prompt template - here is where prompt engineering might improve answers
custom_template = """
{user_input} Please provide a thorough and accurate response, specifically for ApostropheCMS version 3.x. Include code examples when possible. Add validated citations without internal '@' and with `.html` endings from the documentation site. There is no need to comment on this prompt in your answer.
"""

# Create the ChatPromptTemplate object
prompt_template = ChatPromptTemplate.from_template(custom_template)

model = ChatOpenAI(model_name="gpt-4-1106-preview")  # switch to 'gpt-4' or 'gpt-3.5-turbo'

qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

Ask questions

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

# Wrap our vectorstore
compressor = LLMChainExtractor.from_llm(model)

compression_retriever = ContextualCompressionRetriever(
  base_compressor=compressor,
  base_retriever=retriever
)

user_question = input("What is your question? ")
chat_history = []

# Apply the custom template to the question
formatted_question = prompt_template.format(user_input=user_question)

result = compression_retriever.get_relevant_documents(formatted_question)

#result = qa({"question": formatted_question, "chat_history": chat_history})
#chat_history.append((user_question, result["answer"]))
print(f"## -> **Question**: {user_question} \n")
#print(f"### **Answer**: {result['answer']} \n")
pretty_print_docs(result)

/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homeb

## -> **Question**: How would I add a new page type to my project? 

Document 1:

In an Apostrophe project, pages provide a way to show static content, as well as dynamic content delivered by both widgets and pieces. The `@apostrophecms/page-type` module allows for the creation of page types. The selection of these page types by a user tells Apostrophe what template to use to render the page, what content types can be added to a page by the editor through the field schema, as well as what additional dynamic content should be added. You can have as many or as few page types as needed for your site.
----------------------------------------------------------------------------------------------------
Document 2:

```js
// modules/default-page/index.js
module.exports = {
  extend: '@apostrophecms/page-type'
};
```

```js
// app.js
require('apostrophe')({
  shortName: 'my-website',
  modules: {
    'default-page': {}
  }
});
```
---------------------------------------------------------------